## 1. Setting and Preparing ##

In [1]:
!git clone https://github.com/FlagOpen/FlagEmbedding.git

Cloning into 'FlagEmbedding'...
remote: Enumerating objects: 11099, done.
remote: Counting objects: 100% (192/192), done.
remote: Compressing objects: 100% (113/113), done.
remote: Total 11099 (delta 115), reused 98 (delta 79), pack-reused 10907 (from 2)
Receiving objects: 100% (11099/11099), 51.14 MiB | 30.78 MiB/s, done.
Resolving deltas: 100% (6046/6046), done.


In [2]:
%cd FlagEmbedding
!pip install .[finetune]
!pip install deepspeed==0.15.4 
# Notebook gốc ghi deepspeed==0.15.4, hãy đảm bảo bạn dùng phiên bản tương thích
# hoặc nếu có lỗi, thử với phiên bản mà FlagEmbedding khuyến nghị tại thời điểm bạn chạy.

/kaggle/working/FlagEmbedding
Processing /kaggle/working/FlagEmbedding
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 21.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 89.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 69.6 MB/s eta 0:00:00
   ━━━━

In [3]:
!pip install transformers

In [4]:
import json
import torch
import os

In [5]:
if torch.cuda.is_available():
    print(f"GPU is available: {torch.cuda.get_device_name(0)}")
    print(f"Number of GPUs: {torch.cuda.device_count()}")
else:
    print("GPU is NOT available.")

GPU is available: Tesla T4
Number of GPUs: 2


In [6]:
def convert_custom_data_to_flagembedding_jsonl(input_json_path, output_jsonl_path):
    """
    Chuyển đổi dữ liệu JSON tùy chỉnh sang định dạng JSONL mà FlagEmbedding M3 finetuning mong đợi.

    Args:
        input_json_path (str): Đường dẫn đến file JSON đầu vào.
                               Giả định file chứa một danh sách các đối tượng,
                               mỗi đối tượng có "query_caption", "positive_chunks", và "negative_chunks".
        output_jsonl_path (str): Đường dẫn để lưu file JSONL đầu ra.
    """
    print(f"Bắt đầu chuyển đổi file: {input_json_path}")
    items_processed_count = 0
    items_skipped_count = 0
    try:
        with open(input_json_path, 'r', encoding='utf-8') as infile, \
             open(output_jsonl_path, 'w', encoding='utf-8') as outfile:

            # Tải toàn bộ dữ liệu JSON.
            # Giả định rằng file JSON đầu vào của bạn là một danh sách (JSON array)
            # chứa các dictionary "training_triplets_batch" của bạn.
            try:
                custom_data_list = json.load(infile)
            except json.JSONDecodeError as e:
                print(f"Lỗi: Không thể giải mã JSON từ file {input_json_path}. Lỗi: {e}")
                print("Hãy đảm bảo file là một JSON array hợp lệ, ví dụ: [{\"key\": \"value\"}, ...]")
                return

            if not isinstance(custom_data_list, list):
                # Nếu file JSON của bạn không phải là một list ở cấp độ gốc,
                # ví dụ, nó là một dictionary chứa list dữ liệu trong một key cụ thể (ví dụ: {"data": [...]}),
                # bạn cần điều chỉnh ở đây để truy cập vào list đó.
                # Ví dụ: custom_data_list = custom_data_list.get("your_data_key", [])
                print(f"Cảnh báo: Dữ liệu JSON đầu vào không phải là một danh sách (list). Nó là một {type(custom_data_list)}.")
                print("Script này giả định cấu trúc JSON là một danh sách các mục huấn luyện.")
                print("Nếu dữ liệu của bạn nằm trong một key của JSON object, vui lòng điều chỉnh script này hoặc tiền xử lý file JSON của bạn.")
                # Để thử xử lý trường hợp file chỉ chứa một object đơn lẻ (không phải list)
                if isinstance(custom_data_list, dict) and "query_caption" in custom_data_list:
                    print("Dường như file chứa một bản ghi đơn lẻ. Sẽ xử lý nó như một danh sách một mục.")
                    custom_data_list = [custom_data_list]
                else:
                    print(f"Lỗi: Không thể xử lý định dạng JSON đầu vào. Kết thúc chuyển đổi.")
                    return


            for item_index, item in enumerate(custom_data_list):
                query_caption = item.get("query_caption")
                positive_chunks_data = item.get("positive_chunks", []) # list {text, score}
                negative_chunks_data = item.get("negative_chunks", []) # list {text, score, article_id}

                if not query_caption:
                    print(f"Cảnh báo: Bỏ qua mục {item_index} do thiếu 'query_caption': {item}")
                    items_skipped_count += 1
                    continue

                # Trích xuất văn bản từ positive_chunks.
                # FlagEmbedding thường mong đợi một danh sách các positive passages.
                # Ở đây, chúng ta sẽ lấy văn bản từ chunk đầu tiên trong "positive_chunks".
                # Nếu bạn muốn sử dụng tất cả 5 chunks, bạn có thể đưa tất cả vào,
                # script huấn luyện có thể chọn một trong số đó.
                # positive_texts = []
                # if positive_chunks_data:
                #     # Lấy text của chunk đầu tiên làm positive chính
                #     first_positive_chunk = positive_chunks_data[0]
                #     if isinstance(first_positive_chunk, dict) and "text" in first_positive_chunk:
                #         positive_texts.append(first_positive_chunk["text"])
                #     else:
                #         print(f"Cảnh báo: Bỏ qua mục {item_index} cho query '{query_caption[:50]}...' do positive_chunk đầu tiên không hợp lệ: {first_positive_chunk}")
                #         items_skipped_count += 1
                #         continue
                # else:
                #     print(f"Cảnh báo: Bỏ qua mục {item_index} cho query '{query_caption[:50]}...' do thiếu 'positive_chunks'.")
                #     items_skipped_count += 1
                #     continue

                # if not positive_texts: # Phòng trường hợp positive_texts rỗng dù đã qua kiểm tra
                #     print(f"Cảnh báo: Bỏ qua mục {item_index} cho query '{query_caption[:50]}...' do không trích xuất được positive text.")
                #     items_skipped_count += 1
                #     continue

                positive_texts = []
                if positive_chunks_data:
                    valid_pos_found = False
                    for pos_chunk_idx, pos_chunk in enumerate(positive_chunks_data):
                        if isinstance(pos_chunk, dict) and "text" in pos_chunk:
                            positive_texts.append(pos_chunk["text"])
                            valid_pos_found = True
                        else:
                            print(f"Cảnh báo: Positive_chunk không hợp lệ tại vị trí {pos_chunk_idx} cho query '{query_caption[:50]}...': {pos_chunk}. Sẽ bỏ qua chunk này.")
                    
                    if not valid_pos_found: # Nếu lặp qua hết mà không có positive hợp lệ nào
                        print(f"Cảnh báo: Bỏ qua mục {item_index} cho query '{query_caption[:50]}...' do không có positive_chunk nào hợp lệ trong danh sách được cung cấp.")
                        items_skipped_count += 1
                        continue
                else:
                    print(f"Cảnh báo: Bỏ qua mục {item_index} cho query '{query_caption[:50]}...' do thiếu 'positive_chunks'.")
                    items_skipped_count += 1
                    continue

                # Trích xuất văn bản từ negative_chunks
                negative_texts = []
                for neg_chunk_index, neg_chunk in enumerate(negative_chunks_data):
                    if isinstance(neg_chunk, dict) and "text" in neg_chunk:
                        negative_texts.append(neg_chunk["text"])
                    else:
                        print(f"Cảnh báo: Negative_chunk không hợp lệ tại vị trí {neg_chunk_index} cho query '{query_caption[:50]}...': {neg_chunk}")

                # Script của FlagEmbedding thường cần cả positive và negative passages.
                # Nếu không có negative_texts, script có thể dựa vào "in-batch negatives".
                # Tuy nhiên, việc cung cấp negatives rõ ràng thường tốt hơn.
                if not negative_texts:
                    print(f"Thông tin: Không tìm thấy negative_texts tường minh cho query: '{query_caption[:50]}...'. "
                          "Script huấn luyện có thể sẽ sử dụng in-batch negatives.")
                    # Bạn có thể quyết định bỏ qua nếu không có negative tường minh,
                    # tùy thuộc vào chiến lược fine-tuning và khả năng của script FlagEmbedding.
                    # Với mục đích chuyển đổi, chúng ta sẽ cho phép danh sách negative_texts rỗng.

                # Tạo đối tượng JSON cho dòng hiện tại trong file .jsonl
                # Định dạng mong đợi: {"query": str, "pos": list[str], "neg": list[str]}
                output_record = {
                    "query": query_caption,
                    "pos": positive_texts,  # Danh sách chứa văn bản của positive chunk (hoặc các chunks)
                    "neg": negative_texts   # Danh sách các văn bản từ negative chunks
                }
                outfile.write(json.dumps(output_record, ensure_ascii=False) + "\n")
                items_processed_count += 1

            print(f"Hoàn tất chuyển đổi.")
            print(f"Tổng số mục đã xử lý và ghi ra file: {items_processed_count}")
            print(f"Tổng số mục bị bỏ qua: {items_skipped_count}")
            print(f"File JSONL đã được lưu tại: {output_jsonl_path}")

    except FileNotFoundError:
        print(f"Lỗi: Không tìm thấy file đầu vào tại {input_json_path}")
    except Exception as e:
        print(f"Đã xảy ra lỗi không mong muốn trong quá trình chuyển đổi: {e}")
        import traceback
        traceback.print_exc()

In [7]:
input_json_file = "/kaggle/input/1-train-set-for-bge-m3-langchain-1000-2000-local/training_triplets_bm25_top_n_LangChain_1000_2000_local_pos.json"
prepared_training_jsonl_file = "/kaggle/working/training_data_for_flagembedding.jsonl"

print(f"Chuẩn bị chuyển đổi dữ liệu từ: {input_json_file}")
print(f"Dữ liệu sau chuyển đổi sẽ được lưu tại: {prepared_training_jsonl_file}")
convert_custom_data_to_flagembedding_jsonl(input_json_file, prepared_training_jsonl_file)
print("Quá trình chuẩn bị dữ liệu hoàn tất.")

Chuẩn bị chuyển đổi dữ liệu từ: /kaggle/input/1-train-set-for-bge-m3-langchain-1000-2000-local/training_triplets_bm25_top_n_LangChain_1000_2000_local_pos.json
Dữ liệu sau chuyển đổi sẽ được lưu tại: /kaggle/working/training_data_for_flagembedding.jsonl
Bắt đầu chuyển đổi file: /kaggle/input/1-train-set-for-bge-m3-langchain-1000-2000-local/training_triplets_bm25_top_n_LangChain_1000_2000_local_pos.json
Hoàn tất chuyển đổi.
Tổng số mục đã xử lý và ghi ra file: 3229
Tổng số mục bị bỏ qua: 0
File JSONL đã được lưu tại: /kaggle/working/training_data_for_flagembedding.jsonl
Quá trình chuẩn bị dữ liệu hoàn tất.


## II. Finetuned ##

In [8]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
wandb_api_key = user_secrets.get_secret("WANDB_API_KEY")
os.environ["WANDB_API_KEY"] = wandb_api_key

In [9]:
!torchrun --nproc_per_node 2 \
    -m FlagEmbedding.finetune.embedder.encoder_only.m3 \
    --model_name_or_path BAAI/bge-m3  \
    --cache_dir /kaggle/working/cache/model \
    --train_data /kaggle/working/training_data_for_flagembedding.jsonl \
    --cache_path /kaggle/working/cache/data \
    --train_group_size 2 \
    --query_max_len 128 \
    --passage_max_len 1000 \
    --pad_to_multiple_of 8 \
    --same_dataset_within_batch True \
    --small_threshold 0 \
    --drop_threshold 0 \
    --output_dir /kaggle/working/my_finetuned_bge_m3_legal \
    --overwrite_output_dir \
    --learning_rate 2e-5 \
    --bf16 \
    --num_train_epochs 3 \
    --per_device_train_batch_size 1 \
    --dataloader_drop_last True \
    --warmup_ratio 0.1 \
    --gradient_checkpointing \
    --deepspeed /kaggle/working/FlagEmbedding/examples/finetune/ds_stage0.json \
    --logging_steps 20 \
    --weight_decay 0.01 \
    --negatives_cross_device \
    --temperature 0.02 \
    --normalize_embeddings True \
    --unified_finetuning True \
    --use_self_distill True \
    --fix_encoder False \
    --self_distill_start_step 0 \
    --gradient_accumulation_steps 8

W0610 13:58:41.720000 392 torch/distributed/run.py:792] 
W0610 13:58:41.720000 392 torch/distributed/run.py:792] *****************************************
W0610 13:58:41.720000 392 torch/distributed/run.py:792] Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
W0610 13:58:41.720000 392 torch/distributed/run.py:792] *****************************************
2025-06-10 13:58:48.383922: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-06-10 13:58:48.383937: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749563928.409074     395 cuda_dnn.cc:8310] Unabl